<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" align="left" src="https://i.creativecommons.org/l/by-nc-sa/4.0/80x15.png" /></a>&nbsp;| [Emmanuel Rachelson](https://personnel.isae-supaero.fr/emmanuel-rachelson?lang=en) | <a href="https://erachelson.github.io/RLclass_MVA/">https://erachelson.github.io/RLclass_MVA/</a>

<div style="font-size:22pt; line-height:25pt; font-weight:bold; text-align:center;">Chapter 5: Policy gradient algorithms</div>

In this chapter we first depart from the (approximate) dynamic programming framework we have worked with so far. We turn back to a criterion for optimality we introduced in the first chapters, namely the average value of a policy across (starting) states. We explore the question of writing the gradient of this criterion with respect to policy parameters, which leads us to the very important policy gradient theorem and the family of policy gradient algorithms.

<div class="alert alert-success">

**Learning outcomes**   
By the end of this chapter, you should be able to:
- recall and explain the policy gradient theorem
</div>

# Policy gradient methods

**Bottomline question:**   
The previous chapters have focussed on *action-value methods*; they aimed at estimating $Q^*$ in order to deduce $\pi^*$, or they jointly optimized $Q$ and $\pi$. Could we directly optimize $\pi$?

Suppose we have a policy $\pi_\theta$ parameterized by a vector $\theta$. Our goal is to find the parameter $\theta^*$ corresponding to $\pi^*$.

Remarks:
- $\pi_\theta$ might not be able to represent $\pi^*$. We will take a shortcut and call $\pi^*$ the best policy among the $\pi_\theta$ ones.
- for discrete state and action space, the tabular policy representation is a special case of policy parameterization.
- policy parameterization is a (possibly useful) way of introducing prior knowledge on the set of the desired policies.
- the optimal deterministic policies might not belong to the policy subspace of $\pi_\theta$, thus it makes sense to consider stochastic policies for $\pi_\theta$.
- for problems with significant policy approximation, the best approximate policy (among $\pi_\theta$ ones) may very well be stochastic.
- it makes even more sense to consider stochastic policies that it opens the family of environments that we can tackle, like partially observable MDPs or multi-player games.

For stochastic policies, we shall write $\pi_\theta(a|s)$.

In the remainder of the chapter, we will assume that $\pi_\theta$ is differentiable with respect to $\theta$.

Suppose now we define some performance metric $J(\pi_\theta) = J(\theta)$. If $J$ is differentiable and a stochastic estimate $\nabla_\theta J(\theta)$ of the gradient is available, then we can define the gradient ascent update procedure:
$$\theta \leftarrow \theta + \alpha \nabla_\theta J(\theta).$$

We will call **policy gradient methods** all methods that follow such a procedure (whether or not they also learn a value function or not).

Remarks: 
- Note that $J$ is a generic criterion. For example, $J$ could be defined as the value of a starting state (or a distribution of starting states), or as the undiscounted reward over a certain horizon, or as the average reward.
- Why is it interesting to look at policy gradient methods? Because for continuous actions there is no maximization step ($\max_a Q(s,a)$) during evaluation but only a call to $\pi_\theta(s)$ (or a draw from $\pi_\theta(a|s)$). This makes Policy Gradient a method of choice for continuous actions domains (especially common in Robotics).
- When do policy gradient approaches outperform value-based ones? It's hard to give a precise criterion; it really depends on the problem. One thing that comes into play is how easy it is to approximate the optimal policy or the optimal value function. If one is simpler than the other (by "simpler", we mean "it is easier to find a parameterization whose spanned function space almost includes the function to approximate"), then it is a good heuristic to try to approximate it. But this criterion might itself be hard to assess.


**Notations**

- We consider probability density functions $p(X)$ for all random variables $X$.
- For a policy $\pi_\theta$ and a random variable $X$ we write indifferently $p(X|\pi_\theta) = p(X|\theta)$.
- A trajectory is noted $\tau = (s_t,a_t)_{t\in [0,\infty]}$.
- The state random variable at step $t$ is $S_t$ and its law's density is $p_t(s)$.
- The action random variable at step $t$ is $A_t$.

# The policy gradient theorem

## Reminder on the policy optimization objective

Recal the policy optimization objective defined in previous chapters, given a distribution $p_0$ on states:  
$$J(\pi) = \mathbb{E}_{s \sim p_0} \left[ V^{\pi} (s) \right].$$
Or equivalently:  
$$J(\pi) = \mathbb{E}_{(s_i,a_i)_{i \in [0,\infty]}} \left[ \sum_{t=0}^\infty \gamma^t r(s_t,a_t)  | \pi \right].$$
We can switch the sum and the expectation and get:  
$$J(\pi) = \sum_{t=0}^\infty \gamma^t \mathbb{E}_{(s_i,a_i)_{i \in [0,\infty]}} \left[ r(s_t,a_t)  | \pi \right].$$
But $\mathbb{E}_{(s_i,a_i)_{i \in [0,\infty]}} \left[ r(s_t,a_t)  | \pi \right] = \mathbb{E}_{s_t,a_t} \left[ r(s_t,a_t)  | \pi \right]$. So:
$$J(\pi) = \sum_{t=0}^\infty \gamma^t \mathbb{E}_{s_t,a_t} \left[ r(s_t,a_t)  | \pi \right].$$
Now let's introduce the density of $(s_t,a_t)$:
$$J(\pi) = \sum_{t=0}^\infty \gamma^t \int_S \int_A r(s_t,a_t) p(s_t,a_t|\pi) ds_t da_t.$$
But $p(s_t,a_t|\pi) = p(s_t|\pi) p(a_t|s_t,\pi)$. By definition, $p(s_t|\pi) = p_t(s|\pi)$ and $p(a_t=a|s_t=s,\pi) = \pi(a|s)$. So:
$$J(\pi) = \sum_{t=0}^\infty \gamma^t \int_S \int_A r(s,a) p_t(s|\pi) \pi(a|s) ds da.$$
Let us isolate the terms that concern only states:
$$J(\pi) = \int_S \left[ \int_A r(s,a) \pi(a|s) da \right] \sum_{t=0}^\infty \gamma^t p_t(s|\pi) ds.$$
Let's note $\rho^\pi(s) = \sum_{t=0}^\infty \gamma^t p_t(s|\pi)$. We have encountered this quantity in previous chapters and called it the state occupancy measure under policy $\pi$ and starting distribution $p_0$. It is not a proper distribution per se (it sums to $\frac{1}{1-\gamma}$), and is sometimes also called the *improper state distribution under policy $\pi$*. Then we have:
$$J(\pi) = \int_S \left[ \int_A r(s,a) \pi (a|s) da \right] \rho^\pi(s) ds.$$
And so finally, with a slight notation abuse because $\rho^\pi$ is not a probability distribution:
$$J(\theta) = \mathbb{E}_{\substack{s\sim\rho^\pi \\ a\sim \pi}} \left[ r(s,a) \right].$$

In plain words, the value of a policy $\pi$ is the average value of the rewards when states are sampled according to $\rho^\pi$ and actions are sampled according to $\pi$.

## The policy gradient theorem

The crucial problem of computing $\nabla_\theta J(\theta)$ lies in the fact that when $\theta$ changes, both $\pi$ and $\rho^\pi$ change in turn. So there seems to be no straighforward way of evaluating this gradient. One could fall back on a *finite differences* approach to estimating this gradient, but this would require trying out a series of increments $\Delta \theta$ which quickly becomes impractical (because the increment size is hard to tune, especially in stochastic systems, and also because of the sample inefficiency of the approach).

Remark:
- Let's not discard finite difference methods too quickly. They have their merits and showed great successes through methods such as PEGASUS (Ng and Jordan, 2000).

Similarly, one could turn to gradient-free methods, like evolutionary strategies, to search for a maximum of $J(\theta)$.

Remark:
- This topic, which we wont' cover in this chapter, has received renewed attention lately. The interested reader might check the following papers  
[Evolutionary strategies as a scalable alternative to reinforcement learning](todo),  
[Canonical ES](todo),  
[Sigaud's survey](todo).  

The key result of this chapter is that one can express the gradient of $J(\theta)$ as directly proportional to the value of $Q^\pi$ and the gradient of $\pi$:
<div class="alert alert-success">

**Policy gradient theorem:**  
$$\nabla_\theta J(\theta) \propto \mathbb{E}_{\substack{s\sim\rho^\pi \\ a\sim \pi}} \left[ Q^\pi(s,a) \nabla_\theta \log\pi(a|s)\right]$$
</div>

The proof of this result is simple but a bit tedious. We can however give the general intuition. Let's consider trajectories $\tau = (s_0,a_0,r_0,...)$ drawn according to $\pi$ from the starting state. Each of these trajectories has an overall payoff of $G(\tau) = \sum_t \gamma^t r_t$ and is drawn with probability density $p(\tau|\theta)$. Then the objective function can be written:
\begin{align}
J(\theta) &= \mathbb{E}_\tau \left[ G(\tau) | \theta \right]\\
 &= \int G(\tau) p(\tau | \theta) d\tau
\end{align}

So the objective function's gradient is:
\begin{align}
\nabla_\theta J(\theta) &= \nabla_\theta \int G(\tau) p(\tau|\theta) d\tau,\\
 &= \int G(\tau) \nabla_\theta p(\tau|\theta) d\tau,\\
 &= \int G(\tau) p(\tau|\theta) \frac{\nabla_\theta p(\tau|\theta)}{p(\tau|\theta)} d\tau,\\
 &= \mathbb{E}_\tau \left[ G(\tau) \nabla_\theta \log p(\tau|\theta) \right].
\end{align}

Let us study a little the $\nabla_\theta \log p(\tau|\theta)$ term along a series of remarks.

**Remark 1: law of $s_{t+1},a_{t+1}$ given the policy and history.**  
One has $p(s_{t+1},a_{t+1} | (s_i,a_i)_{i \in [0,t]}, \theta) = p(s_{t+1} | (s_i,a_i)_{i \in [0,t]}, \theta) p(a_{t+1} | s_{t+1}, (s_i,a_i)_{i \in [0,t]}, \theta)$.  
But the transition model is Markovian, so $p(s_{t+1} | (s_i,a_i)_{i \in [0,t]}, \theta) = p(s_{t+1} | s_t, a_t)$.  
And the law of $a_{t+1}$ is given by the policy, so $p(a_{t+1} | s_{t+1}, (s_i,a_i)_{i \in [0,t]}, \theta) = \pi_\theta(a_{t+1}|s_{t+1})$.  
Consequently:
$$p(s_{t+1},a_{t+1} | (s_i,a_i)_{i \in [0,t]}, \theta) = p(s_{t+1} | s_t, a_t) \pi_\theta(a_{t+1}|s_{t+1}).$$

**Remark 2: probability density of a trajectory.**  
Recall that $p(\tau|\theta) = p((s_t,a_t)_{t\in [0,\infty]}|\theta)$.  
This joint probability can be decomposed into conditional probabilities:  
$p(\tau|\theta) = p(s_0,a_0|\theta) \prod_{t=0}^\infty p(s_{t+1},a_{t+1} | (s_i,a_i)_{i \in [0,t]}, \theta)$.
By the previous remarks allows us to simplify to:
$p(\tau|\theta) = p(s_0,a_0|\theta) \prod_{t=0}^\infty p(s_{t+1} | s_t, a_t) \pi_\theta(a_{t+1}|s_{t+1})$.
By expanding the first term into $p(s_0)\pi_\theta(a_0|s_0)$ and reordering the terms inside the product, we obtain:
$$p(\tau|\theta) = p(s_0) \prod_{t=0}^\infty p(s_{t+1} | s_t, a_t) \pi_\theta(a_t|s_t).$$

**Remark 3: the grad-log-prob trick.**  
Now let us consider the full $\nabla_\theta \log p(\tau|\theta)$ term. The previous remarks tell us that  
$$\nabla_\theta \log p(\tau|\theta) = \nabla_\theta \log p(s_0) + \sum_{t=0}^\infty \left[ \nabla_\theta \log p(s_{t+1} | s_t, a_t) + \nabla_\theta \log \pi_\theta(a_t|s_t)\right].$$
But the initial state distribution and the transition model do not depend on $\theta$, so this expression boils down to:
$$\nabla_\theta \log p(\tau|\theta) = \sum_{t=0}^\infty \nabla_\theta \log \pi_\theta(a_t|s_t).$$

And we will admit the step which leads to:
$$\nabla_\theta J(\theta) = \mathbb{E}_{\substack{s\sim\rho^\pi \\ a\sim \pi}} \left[ G(\tau) \nabla_\theta \log \pi_\theta(a|s) \right].$$

TODO REMOVE THIS PART, the Q^\pi = G is not trivial.  
And finally, since $Q^\pi(s,a) = \mathbb{E} [G(\tau) | S_0=s, A_0=a, \theta]$, we obtain that:
$$\nabla_\theta J(\theta) \propto \mathbb{E}_{\substack{s\sim\rho^\pi \\ a\sim \pi}} \left[ Q^\pi(s,a) \nabla_\theta \log\pi(a|s)\right].$$

**Interpretation**  
Let us consider a fixed $s$.  
Then $\nabla_\theta \pi(a|s)$ is the answer to "in what direction should I change $\theta$ to increase the probability of taking action $a$ in $s$?".  
The policy gradient theorem tells us that in order to improve the value of $\pi_\theta$, we should change $\theta$ in a direction that is a linear combination of all $\nabla_\theta \pi(a|s)$, giving more weight to actions that provide a large $Q^\pi(s,a)$. 

TODO EXERCISE IMPLEMENT VPG  
big warning about the loss

# REINFORCE

# Actor-Critic

# TRPO

# PPO

bonus:  
- ES for RL
- GAE
- 